# 1. Import Data

In [ ]:
import pandas as pd

players_path = '../data/players.csv'
players_valuations_path = '../data/player_valuations.csv'

players = pd.read_csv(players_path)
players_valuations = pd.read_csv(players_valuations_path)

# 2. Dataset Information

## 2-1. players

In [ ]:
print(players.info())
display(players.head())
print(players.columns)

## 2-3. players_valuations

In [ ]:
print(players_valuations.info())
display(players_valuations.head())
print(players_valuations.columns)

## 결측치 확인

In [ ]:
print(players.isna().sum(),'\n')
print(players_valuations.isna().sum())

## 기초 통계량

In [ ]:
# 특정 컬럼에 대해 unique 값 개수 출력 함수 정의
def count_value(df, column):
    count = len(df[column].unique())
    print(f'Total {column}: {count}')

columns = ['player_id', 'current_club_id', 'country_of_citizenship'] # player_id, 소속 구단, 국적에 대한 unique 개수 확인

for column in columns:
    count_value(players, column)

In [ ]:
# 선수 시장가치(market_value_in_eur)의 기초 통계량
players_valuations['market_value_in_eur'].describe()

In [ ]:
pd.options.display.float_format = '{:.0f}'.format
players_valuations['market_value_in_eur'].describe()

In [ ]:
# 평균 시장가치 이상인 선수 비율 계산
mean_ = players_valuations['market_value_in_eur'].mean()
over_mean = len(players_valuations[players_valuations['market_value_in_eur'] > mean_])
total = len(players_valuations)
print(f"percentile of player over mean value: {over_mean/total*100:.2f}%")

# 3. Handling DataFrame

In [ ]:
players_with_val = pd.merge(players, players_valuations, on='player_id')
players_with_val[players_with_val['last_name']=='Son'].tail()

## 3-1. 태어난 년도 기준으로 각 연봉을 받았던 때의 나이 계산하여 age 컬럼 추가

In [ ]:
# "YYYY-MM-DD” 형식의 date에서 연도만 뽑아내서 dateyear 컬럼 추가
players_with_val['dateyear'] = players_with_val['date'].apply(lambda x: int(str(x)[:4]) if pd.notna(x) else None)
players_with_val['age'] = players_with_val['dateyear'] - players_with_val['date_of_birth'].apply(
    lambda x: int(str(x)[:4]) if pd.notna(x) else None
)

In [ ]:
# 선수별 동일 연도 데이터 중복 제거 → 가장 마지막 기록만 남김
players_with_val.drop_duplicates(['player_id','dateyear'], keep='last',inplace=True)
players_with_val[players_with_val['last_name']=='Son'].head()

In [ ]:
# 분석에 필요한 컬럼만 선별
columns = [
    'player_id', 'current_club_id_y', 'first_name', 'last_name', 'name', 'last_season_x', 'country_of_citizenship', 'city_of_birth', 'position', 'sub_position', 'dateyear', 'age', 'market_value_in_eur_y']
players_with_val = players_with_val[columns]

players_with_val.rename(
    columns={
    "current_club_id_y": "current_club_id",
    "last_season_x": "last_season",
    "market_value_in_eur_y":"market_value_in_eur"
    },
    inplace=True
)

players_with_val

In [ ]:
players_with_val['market_value_in_eur'].describe()

In [ ]:
# 2022년 기준 데이터만 추출
players_with_val_2022 = players_with_val[(players_with_val['dateyear'] == 2022) & (players_with_val['last_season'] == 2022)]
players_with_val_2022

In [ ]:
# 2022년 선수별 market_value 순위 계산
players_with_val_2022['market_value_rank'] = players_with_val_2022['market_value_in_eur'].rank(method="min", ascending=False)

# market_value 기준 정렬 후 손흥민 선수 확인
players_with_val_2022.sort_values(by='market_value_rank')
players_with_val_2022[players_with_val_2022['last_name'] == 'Son']

# 4. Visaulization

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# 한글 폰트 설정
mpl.rc('font', family='AppleGothic')
# 음수 기호 깨짐 방지
mpl.rcParams['axes.unicode_minus'] = False

## 4-1. 연도별 선수가치 분포도

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(8,6))
plt.boxplot(players_with_val['market_value_in_eur'])
plt.title('선수들의 시장가치 분포도')
plt.show()

대부분의 선수들의 시장가치는 낮은 구간에 몰려있고
극소수의 스타 선수들만 박스플롯의 이상치로 표시되어있음을 볼 수 있다

In [ ]:
# 선수 가치별로 그룹화하여 같은 시장가치를 가진 선수 수를 카운트
plt.figure(figsize=(8,6))
players_with_val.groupby('market_value_in_eur')['player_id'].count().plot()
plt.title('가치별 선수 수 분포도')
plt.show()

* x축: market_value_in_eur (시장가치, 보통 유로 단위).
*  같은 시장가치를 가진 선수 수.
결과적으로 “특정 시장가치를 가진 선수가 몇 명 있는지”를 보여줌.

**문제점**
* 시장가치 데이터는 연속형(금액이 다양함)인데, 그대로 groupby 하면 x축에 값이 너무 많아 복잡하게 나옴.
* 예: 시장가치가 500,000 유로, 510,000 유로, 520,000 유로… 이런 값들이 전부 개별 x축으로 들어감 → 해석 어려움.

In [ ]:
sum_per_year = players_with_val.groupby('dateyear')['market_value_in_eur'].sum()
x = sum_per_year.index
y = sum_per_year.values

plt.figure(figsize=(8,6))
plt.plot(x, y)
plt.xticks(rotation=45)
plt.title('연도별 전체 선수 시장가치 총합의 변화 추세')
plt.show()

In [ ]:
max_per_year = players_with_val.groupby('dateyear')['market_value_in_eur'].max()
x = max_per_year.index
y = max_per_year.values

plt.figure(figsize=(8,6))
plt.plot(x,y)
plt.xticks(rotation=45)
plt.title('연도별 선수가치 max 값의 변화 추세')
plt.show()

In [ ]:
mean_per_year = players_with_val.groupby('dateyear')['market_value_in_eur'].mean()
x = mean_per_year.index
y = mean_per_year.values

plt.figure(figsize=(8,6))
plt.plot(x, y)
plt.xticks(rotation=45)
plt.title('연도별 선수가치 평균값의 변화 추세')
plt.show()

In [ ]:
y = players_with_val.groupby('dateyear')['player_id'].count()

plt.figure(figsize=(8,6))
plt.bar(x, y)
plt.xticks(rotation=45)
plt.title('연도별 선수가치별 선수 수의 변화 추세')
plt.show()

In [ ]:

colors = ['blue' if count > 5000 else 'green' for count in y]

plt.figure(figsize=(8,6))
plt.bar(x, y, color=colors)
plt.xticks(rotation=45)
plt.title("연도별 선수 시장가치 총합")
plt.show()


In [ ]:
players_with_val = players_with_val[(players_with_val['dateyear'] >= 2013) & (players_with_val['dateyear'] < 2023)]

In [ ]:
years = sorted(players_with_val['dateyear'].unique())

num_plots = len(years)
num_rows = 4
num_cols = (num_plots + 3) // 4

fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 8))

axes = axes.flatten()

for i, year in enumerate(years):
    market_values = players_with_val[players_with_val['dateyear'] == year]['market_value_in_eur'].values

    ax = axes[i]
    ax.boxplot(market_values)
    ax.set_title(year)

for j in range(num_plots, num_rows * num_cols):
    fig.delaxes(axes[j])

fig.suptitle("연도별 선수 시장가치 분포", fontsize=16)

plt.tight_layout()

plt.show()

## 4-2. 나이별 시장가치 분포도

In [ ]:
players_with_val.info()

In [ ]:
age_market_values = players_with_val.groupby('age')['market_value_in_eur'].mean()

plt.figure(figsize=(8,6))
age_market_values.plot(kind='bar')
plt.xlabel('Age')
plt.ylabel('Mean Market Value (EUR)')
plt.xticks(rotation=0)
plt.title('나이별 평균가치 분포도')
plt.show()

In [ ]:
filtered_data = players_with_val[players_with_val['age'] <= 35]

age_market_values = filtered_data.groupby('age')['market_value_in_eur'].mean()

sorted_values = age_market_values.sort_values(ascending=False)

top_5_intervals = sorted_values.head(5).index

colors = ['blue' if age in top_5_intervals else 'green' for age in age_market_values.index]

plt.figure(figsize=(8, 6))
age_market_values.plot(kind='bar', color=colors)
plt.xlabel('Age')
plt.ylabel('Mean Market Value (EUR)')
plt.title('나이별 평균가치 분포(~35 세) 중 top 5 ')
plt.xticks(rotation=45)
plt.show()

In [ ]:

import numpy as np

def get_top_name(g):
    if g['market_value_in_eur'].notna().any():
        return g.loc[g['market_value_in_eur'].idxmax(), 'name']
    return np.nan

top_players = (
    filtered_data
    .groupby('age', group_keys=False)
    .apply(get_top_name)
)


plt.figure(figsize=(12, 6))
ax = age_market_values.plot(kind='bar', color=colors)
ax.set_xlabel('Age')
ax.set_ylabel('Mean Market Value (EUR)')
ax.set_title('나이별 최고가 선수 (Up to 35)')

for i, (age, value) in enumerate(zip(age_market_values.index, age_market_values.values)):
    name = top_players.get(age, None)
    if pd.notna(name):
        ax.text(i, value * 1.01, name,
                ha='center', va='bottom',
                fontsize=9,
                fontweight='bold' if age in sorted_values.head().index else 'normal',
                rotation=0)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4-3. 포지셜별 가치 분포

In [ ]:
position_market_values = players_with_val.groupby('position')['market_value_in_eur'].mean()

plt.figure(figsize=(8,6))
position_market_values.plot(kind='bar')
plt.xlabel('Position')
plt.ylabel('Mean Market Value (EUR)')
plt.title('포지션별 평균 가치')
plt.xticks(rotation=45)
plt.show()


In [ ]:
position_market_values = players_with_val.groupby(['position', 'dateyear'])['market_value_in_eur'].mean()

position_market_values = position_market_values.reset_index()

plt.figure(figsize=(12, 8))
for position in position_market_values['position'].unique():
    position_data = position_market_values[position_market_values['position'] == position]
    plt.plot(position_data['dateyear'], position_data['market_value_in_eur'], label=position)

plt.xlabel('Year')
plt.ylabel('Mean Market Value (EUR)')
plt.title('포지션별 평균 시장가치 추이 (연도별)')
plt.legend()
plt.xticks(rotation=45)
plt.show()

In [ ]:
positions = players_with_val['position'].unique()

fig, axes = plt.subplots(len(positions), figsize=(10, 20))

for i, position in enumerate(positions):
    ax = axes[i]
    ax.set_title(position)

    position_data = players_with_val[players_with_val['position'] == position]
    sub_positions = position_data['sub_position'].unique()

    for sub_position in sub_positions:
        sub_position_data = position_data[position_data['sub_position'] == sub_position]
        sub_position_value = sub_position_data.groupby('dateyear')['market_value_in_eur'].mean()

        ax.plot(sub_position_value.index, sub_position_value.values, label=sub_position)

    ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
positions = players_with_val['position'].unique()

fig, axes = plt.subplots(len(positions), figsize=(10, 20))

for i, position in enumerate(positions):
    ax = axes[i]
    ax.set_title(position)

    position_data = players_with_val[players_with_val['position'] == position]
    sub_positions = position_data['sub_position'].unique()

    for sub_position in sub_positions:
        sub_position_data = position_data[position_data['sub_position'] == sub_position]
        sub_position_value = sub_position_data.groupby('dateyear')['market_value_in_eur'].mean()

        ax.plot(sub_position_value.index, sub_position_value.values, label=sub_position)

        top_players = sub_position_data.loc[sub_position_data.groupby('dateyear')['market_value_in_eur'].idxmax(), 'name']

        for year, player in zip(sub_position_value.index, top_players):
            ax.text(year, sub_position_value[year], player, ha='center', va='bottom')

    ax.legend()

plt.tight_layout()
plt.show()


## 4-4. 국가별 가치 분포

In [ ]:
country_player_counts = players_with_val.drop_duplicates('player_id')['country_of_citizenship'].value_counts()
country_player_counts.sort_values(ascending=False)

top_10_countries = country_player_counts.head(10)

plt.figure(figsize=(12, 8))
top_10_countries.plot(kind='bar')
plt.xlabel('Country of Citizenship')
plt.ylabel('Number of Players')
plt.title('	Top 10 국적별 선수 수 분포')

for i, value in enumerate(top_10_countries):
    country = top_10_countries.index[i]
    plt.text(i, value, str(value), ha='center', va='bottom')

plt.xticks(rotation=45)
plt.show()

In [ ]:
country_player_counts_2022 = players_with_val_2022.drop_duplicates('player_id')['country_of_citizenship'].value_counts()
country_player_counts_2022.sort_values(ascending=False)

top_10_countries = country_player_counts_2022.head(10)

plt.figure(figsize=(12, 8))
top_10_countries.plot(kind='bar')
plt.xlabel('Country of Citizenship')
plt.ylabel('Number of Players')
plt.title('	Top 10 국적별 선수 수 분포 - 2022')

for i, value in enumerate(top_10_countries):
    country = top_10_countries.index[i]
    plt.text(i, value, str(value), ha='center', va='bottom')

plt.xticks(rotation=45)
plt.show()

## 4-5. 국가별 선수 수 분포도

In [ ]:
from geopy.geocoders import Nominatim
import time

locations = []

errors = []

geolocator = Nominatim(user_agent="my-app")

countries = country_player_counts.index

for country in countries:
    try:
        location = geolocator.geocode(country)
    except Exception as e:
        print(f"지오코딩 오류 - 나라: {country}. 오류: {e}")
        errors.append(country)
        continue

    time.sleep(0.3)
    latitude = location.latitude
    longitude = location.longitude

    locations.append((country, latitude, longitude))
    print("나라:", country)
    print("위도:", latitude)
    print("경도:", longitude)
    print("-"*20)



In [ ]:
locations_df_2022 = pd.DataFrame(locations)
locations_df_2022.rename(columns={
    0: 'country',
    1: 'Latitude',
    2: 'Longitude'
}, inplace=True)

locations_df_2022['player_count'] = locations_df_2022.country.apply(lambda x: country_player_counts_2022.get(x, 0))

In [ ]:
import plotly.express as px


fig = px.density_mapbox(
    locations_df_2022,
    lat="Latitude",
    lon="Longitude",
    z="player_count",
    radius=15,
    center=dict(lat=20, lon=0),
    zoom=1,
    mapbox_style="open-street-map"
)

fig.update_layout(
    title="Density Map of 2022 Players"
)

fig.show(renderer="browser")

In [ ]:
cities_in_england = players_with_val[players_with_val['country_of_citizenship'] == 'England'].drop_duplicates('player_id')['city_of_birth']
print(f"Number of City in Enlgand: {len(cities_in_england.unique())}")

england_players_count = cities_in_england.value_counts()

top_10_city_in_england = england_players_count.head(10)

plt.figure(figsize=(12, 8))
top_10_city_in_england.plot(kind='bar')
plt.xlabel('City of England')
plt.ylabel('Number of Playesr')
plt.title('영국 도시별 선수 수 분포')

for i, value in enumerate(top_10_city_in_england):
    city = top_10_city_in_england.index[i]
    plt.text(i, value, str(value), ha='center', va='bottom')

plt.xticks(rotation=45)
plt.show()

In [ ]:
top_50_city_in_england = england_players_count.head(50)

enlgand_city_locations = []

errors = []

geolocator = Nominatim(user_agent="my-app")

cities = top_50_city_in_england.index

for city in cities:
    try:
        location = geolocator.geocode(city)
    except Exception as e:
        print(f"지오코딩 오류 - 나라: {city}. 오류: {e}")
        errors.append(city)
        continue

    time.sleep(0.3)
    latitude = location.latitude
    longitude = location.longitude

    enlgand_city_locations.append((city, latitude, longitude))


In [ ]:
england_locations_df = pd.DataFrame(enlgand_city_locations)
england_locations_df.rename(columns={
    0: 'city',
    1: 'Latitude',
    2: 'Longitude'
}, inplace=True)

england_locations_df['player_count'] = england_locations_df.city.apply(lambda x: england_players_count[x])
england_locations_df.head()

In [ ]:
import folium

england_location = [55.8670, -4.2621]
england_map = folium.Map(location=england_location, zoom_start=5)

for index, row in england_locations_df.iterrows():
    city = row['city']
    latitude = row['Latitude']
    longitude = row['Longitude']
    player_count = row['player_count']

    radius = player_count / 5
    color = 'darkred' if player_count > 100 else 'red' if player_count > 50 else 'lightred'
    folium.CircleMarker(
        location=[latitude, longitude],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        tooltip=f"<b>{city}</b><br>Player Count: {player_count}"
    ).add_to(england_map)

england_map

In [ ]:
df_for_animation = pd.DataFrame(
    players_with_val[players_with_val['country_of_citizenship'] == 'England']
    .groupby(['dateyear', 'city_of_birth'])['player_id']
    .count()
    )

df_for_animation.reset_index(inplace=True)
df_for_animation.rename(
    columns={
        'city_of_birth': 'city',
        'player_id': 'player_count'
        },
    inplace=True
    )

df_for_animation = pd.merge(
    df_for_animation, england_locations_df[['city', 'Latitude', 'Longitude']],
    on='city', how='left')

df_for_animation.dropna(inplace=True)
df_for_animation

In [ ]:
import plotly.express as px

fig = px.scatter_geo(
    df_for_animation, lat="Latitude", lon="Longitude",
    color="player_count", size="player_count", hover_name="city",
    animation_frame="dateyear", projection="natural earth"
)

fig.update_geos(
    projection_scale=3.5,
    scope="europe",
    center=dict(lat=55.8670, lon=-4.2621),
)

fig.update_layout(
    title="England Football Player Count by City Over Years",
    height=600, width=800,
)

fig.show(renderer="browser")

## 4-5. 클럽 단위 총 가치

In [ ]:

clubs_path = '../data/clubs.csv'
competitions_path = '../data/competitions.csv'

clubs = pd.read_csv(clubs_path)
competitions = pd.read_csv(competitions_path)

print(clubs.head())
print(competitions.head())


In [ ]:
clubs.info()

In [ ]:
clubs_2022 = clubs[clubs['last_season'] == 2022]

In [ ]:
total_market_values = players_with_val_2022.groupby('current_club_id')['market_value_in_eur'].sum()

clubs_2022['total_market_value'] = clubs_2022['club_id'].apply(lambda club_id: total_market_values[club_id])
clubs_2022

In [ ]:
columns = ['competition_id', 'name', 'country_name']
competitions = competitions[columns].rename(columns={'competition_id': 'domestic_competition_id'})

In [ ]:
clubs_2022 = pd.merge(clubs_2022, competitions, on='domestic_competition_id')

display(clubs_2022.head())
print(clubs_2022.info())

In [ ]:
columns = ['club_id', 'club_code', 'name_x', 'total_market_value', 'squad_size', 'average_age', 'foreigners_number', 'foreigners_percentage', 'name_y', 'country_name']

clubs_2022 = clubs_2022[columns]
clubs_2022.rename(columns={
    "name_x": "club_name",
    "name_y": "competition_name"
}, inplace=True)

clubs_2022

In [ ]:
clubs_2022

In [ ]:
import pandas as pd, plotly
print(pd.__version__, plotly.__version__)

In [ ]:
import plotly.express as px

sorted_clubs_2022 = clubs_2022.sort_values('total_market_value', ascending=False)

fig = px.treemap(
    sorted_clubs_2022,
    path=['club_name'],  # 'name'이 club_name임
    values='total_market_value',
    labels={'club_name': 'Club Name', 'total_market_value': 'Total Market Value'},
    title='Clubs in 2022 by Total Market Value'
)
fig.show()


In [ ]:
sorted_clubs_2022 = clubs_2022.sort_values('total_market_value', ascending=False)

fig = px.treemap(sorted_clubs_2022, path=['country_name', 'club_name'], values='total_market_value',
                 labels={'club_name': 'Club Name', 'total_market_value': 'Total Market Value'},
                 title='Clubs in 2022 by Total Market Value')
fig.show(renderer="browser")

In [ ]:
fig = px.bar(sorted_clubs_2022, x='club_name', y='total_market_value', color='country_name',
             labels={'club_name': 'Club Name', 'total_market_value': 'Total Market Value'},
             title='Clubs in 2022 by Total Market Value')
fig.show(renderer="browser")

In [ ]:
fig = px.box(clubs_2022, x='competition_name', y='total_market_value', color='competition_name',
             title='Market Value Distribution by Competition(Domestic League)',
             labels={'competition_name': 'Competition', 'total_market_value': 'Total Market Value'})
fig.show(renderer="browser")

In [ ]:
fig = px.box(clubs_2022, x='competition_name', y='average_age', color='competition_name',
             title='Age Distribution by Competition(Domestic League)',
             labels={'competition_name': 'Competition', 'average_age': 'Average Age'})
fig.show(renderer="browser")